In [2]:
from keras.layers import Dense, Input, Lambda, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
import keras

Using TensorFlow backend.


In [3]:
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import os
import glob
from scipy.misc import imread,imresize
from keras.applications.vgg16 import VGG16
from skimage import color
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.optimizers import Adam





In [163]:
class ImageDataset:
    def __init__(self,image_weight = 224, image_height = 224):
        self.image_weight = image_weight
        self.image_height = image_height
        self.distort_path = './A/test/images/'#'../../Descargas/ava_downloader/image/'
        self.original_path = './A/test/target/'
        self.start_idx = 0
        self.end_idx = 10
        
        self.input_image = []
        self.input_hist = []
        self.output = []
        
        
        self.input_load()
        self.pred_load()
        
    def input_load(self):
        raw_image_list = glob.glob(self.distort_path + '*.jpg')
        raw_image_list = sorted(raw_image_list)[self.start_idx:self.end_idx]
        print raw_image_list
        
        for i, img_path in enumerate(raw_image_list):
            if i%100 == 0:
                print ("processed %d out of %d" %( i, len(raw_image_list) ))
            try:
                """
                image = imresize(imread(img_path), (self.image_weight,self.image_height))#/255.0-0.5
                #image = image.img_to_array(image)
                #image = np.expand_dims(image)
                #image = image.img_to_array(image, axis = 0)
                image = preprocess_input(image)
                self.input_image.append(image)
                """
                print 'done distorted image', i
                img = image.load_img(img_path, target_size=(224, 224))
                x = image.img_to_array(img)
                #x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                self.input_image.append(x)
                
                
                color_hist = lab_hist(x)
                self.input_hist.append(np.stack(np.array(color_hist)))
                
                #parameters = np.ones(6)
                #image_flat = x.flatten()
                #output = np.concatenate([parameters , image_flat])
                #self.output.append(output)
                
            except Exception as e:
                print str(e)
                
                
    def pred_load(self):
        raw_image_list = glob.glob(self.original_path + '*.jpg')
        raw_image_list = sorted(raw_image_list)[self.start_idx:self.end_idx]
        
        for i, img_path in enumerate(raw_image_list):
            if i%100 == 0:
                print ("processed %d out of %d" %( i, len(raw_image_list) ))
            try:
                """
                image = imresize(imread(img_path), (self.image_weight,self.image_height))#/255.0-0.5
                #image = image.img_to_array(image)
                #image = np.expand_dims(image)
                #image = image.img_to_array(image, axis = 0)
                image = preprocess_input(image)
                self.input_image.append(image)
                """
                print 'done original image', i
                img = image.load_img(img_path, target_size=(224, 224))
                x = image.img_to_array(img)
                 
                
                #x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                #self.input_image.append(x)
               
                
                #color_hist = lab_hist(x)
                #self.input_hist.append(np.stack(np.array(color_hist)))
                
                parameters = np.ones(6)
                image_flat = x.flatten()
                output = np.concatenate([parameters , image_flat])
                self.output.append(output)
                
            except Exception as e:
                print str(e)
                
    
def lab_hist(image_np):
    image_np_rgb = image_np/255.0 # 0~1 rgb
    image_np_lab = color.rgb2lab(image_np_rgb)

    num_bin_L = 10
    num_bin_a = 10
    num_bin_b = 10

    L_max = 100
    L_min = 0
    a_max = 60
    a_min = -60
    b_max = 60
    b_min = -60
    image_np_lab = image_np_lab.reshape([224*224,3])
    H, edges = np.histogramdd(image_np_lab, bins=(num_bin_L, num_bin_a, num_bin_b), \
            range=((L_min, L_max), (a_min, a_max), (b_min, b_max)))
    return H.reshape(1000)/1000.0
        
mydata = ImageDataset()
        

['./A/test/images/000027.jpg__0.052476.jpg', './A/test/images/000047.jpg__0.853644.jpg', './A/test/images/000048.jpg__0.057931.jpg', './A/test/images/000065.jpg__0.000169.jpg', './A/test/images/000096.jpg__1.149314.jpg', './A/test/images/000103.jpg__0.166729.jpg', './A/test/images/000119.jpg__0.180421.jpg', './A/test/images/000129.jpg__0.000262.jpg', './A/test/images/000136.jpg__0.029303.jpg', './A/test/images/000147.jpg__0.069185.jpg']
processed 0 out of 10
done distorted image 0
done distorted image 1
done distorted image 2
done distorted image 3
done distorted image 4
done distorted image 5
done distorted image 6
done distorted image 7
done distorted image 8
done distorted image 9
processed 0 out of 10
done original image 0
done original image 1
done original image 2
done original image 3
done original image 4
done original image 5
done original image 6
done original image 7
done original image 8
done original image 9


In [273]:
class NeuralNetwork:
    def __init__(self, image_weight = 224, image_height = 224):
        self.image_size = (image_weight, image_height,3)
        self.hist_size = 1000
        self.batch_size = 4
        
        optimizer = Adam(0.0002, 0.2)
        
        self.input_image = Input(shape = self.image_size, name='input_image')
        self.feature_NN = self.build_VGG()(self.input_image)
               
        
        self.input_hist = Input(shape = (self.hist_size,), name='input_hist')
        x = keras.layers.concatenate([self.feature_NN, self.input_hist])
        self.parameters_NN = self.build_NN()(x)
        
        z = Flatten()(self.input_image)
        
        #z = Input(shape=(self.image_size))(self.input_image)
        self.output = keras.layers.concatenate([z, self.parameters_NN])

        self.combined = Model([self.input_image, self.input_hist], self.output)
        self.combined.compile(loss = _loss_f, optimizer = optimizer)
        
        
    def build_NN(self, input_size = (5096,)):
        inputs = Input(shape = input_size)
        x = Dense(512, activation = 'relu')(inputs)
        x = Dense(64, activation = 'relu')(x)
        parameters = Dense(6, activation = 'linear')(x)
        model = Model(inputs=inputs, outputs=parameters)
        # model.summary
        return model
        
    def build_VGG(self, istrainable = False):
        model = VGG16(weights = 'imagenet')
        layer_name = 'fc2'
        model2 = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
        model2.trainable = istrainable
        return model2
        
class DeepFeatureNetwork:
    def __init__(self, input_size, model_func, model_path):
        self.input_tensor = tf.placeholder(tf.float32, shape=input_size)
        self.feature, self.weights = model_func(self.input_tensor, model_path)
        
    def init_model(self):
        tf.initialize_variables(self.weights).run()

    def get_feature(self, in_data):
        return sess.run(self.feature, feed_dict={self.input_tensor: in_data+0.5})
    
def _loss_f(y_true, y_pred):
    
    y_pred = y_pred[:,:]
    #parameter_1 = K.slice(y_pred,[0],[1])
    #parameter_2 = K.slice(y_pred,[1],[1])
    #parameter_3 = K.slice(y_pred,[2],[1])
    #parameter_4 = K.slice(y_pred,[3],[1])
    #parameter_5 = K.slice(y_pred,[4],[1])
    #parameter_6 = K.slice(y_pred,[5],[1])
    
    x = K.slice(y_pred, [6],[-1])
    
    image_pred = K.reshape(x,[224,224,3])
    
    x = K.slice(y_true,[6],[-1])
    
    image_true = K.reshape(x,[224,224,3])
    
    # Contrast
    contrast_mean = K.mean(image_pred)
    contrast_degenerate = K.zeros([224,224,3]) + contrast_mean
    
    image_pred = parameter_1 * image_pred + (1-parameter_1)*contrast_degenerate
    image_pred = K.clip(image_pred,0,1)
    
     # Brightness
    #brightness_degenerate = np.zeros(image_pred.shape)
    
    image_pred = parameter_2 * image_pred # + (1 - parameters[1]) * brightness_degenerate
    image_pred = K.clip(image_pred,0,1)
    
    # Color Saturation
    
    saturation_degenerate = K.mean(image_pred,2)
    
    red_layer = K.slice(image_pred,[0,0,0],[-1,-1,1])
    blue_layer = K.slice(image_pred,[0,0,1],[-1,-1,1])
    green_layer = K.slice(image_pred,[0,0,2],[-1,-1,1])
    
    red_layer_ = (parameter_3 * red_layer) + K.expand_dims( (1- parameter_3) * saturation_degenerate, axis= 2)
    blue_layer_ = (parameter_3 * blue_layer) +  K.expand_dims( (1- parameter_3) * saturation_degenerate, axis= 2)
    green_layer_ = (parameter_3 * green_layer) +  K.expand_dims( (1- parameter_3) * saturation_degenerate, axis= 2)
    
    image_pred = K.concatenate ([red_layer_, blue_layer_, green_layer_], axis =2)
    
    image_pred = K.clip(image_pred,0,1)
    
    # White Balance
    red_layer = K.slice(image_pred,[0,0,0],[-1,-1,1])
    blue_layer = K.slice(image_pred,[0,0,1],[-1,-1,1])
    green_layer = K.slice(image_pred,[0,0,2],[-1,-1,1]) 
    
    red_layer_ = (parameter_4 * red_layer) 
    blue_layer_ = (parameter_5 * blue_layer)
    green_layer_ = (parameter_6 * green_layer)
    
    image_pred = K.concatenate ([red_layer_, blue_layer_, green_layer_], axis =2)
    
    image_pred = K.clip(image_pred,0,1)
    
    mse = K.mean(K.square(image_pred - image_true))
    
    return  mse 
        
    
"""    
def _loss_f(y_true, y_pred):
    
        
    parameter_1 = K.slice(y_pred,[0],[1])
    parameter_2 = K.slice(y_pred,[1],[1])
    parameter_3 = K.slice(y_pred,[2],[1])
    parameter_4 = K.slice(y_pred,[3],[1])
    parameter_5 = K.slice(y_pred,[4],[1])
    parameter_6 = K.slice(y_pred,[5],[1])
    
    x = K.slice(y_pred, [6],[-1])
    
    image_pred = K.reshape(x,[224,224,3])
    
    x = K.slice(y_true,[6],[-1])
    
    image_true = K.reshape(x,[224,224,3])
    
    # Contrast
    contrast_mean = K.mean(image_pred)
    contrast_degenerate = K.zeros(224,224,3) + contrast_mean
    
    image_pred = parameter_1 * image_pred + (1-parameter_1)*contrast_degenerate
    image_pred = K.clip(image_pred,0,1)
    
     # Brightness
    #brightness_degenerate = np.zeros(image_pred.shape)
    
    image_pred = parameter_2 * image_pred # + (1 - parameters[1]) * brightness_degenerate
    image_pred = np.clip(image_pred,0,1)
    
    
    x = np.split(y_pred,[6])

x = K.slice(y_pred, [1],[6])
y = K.slice(y_pred, [6],[-1])
z = K.reshape(y_pred,[3,3,3])
    
    
    parameters = x[0]
    image_pred = x[1].reshape(224,224,3)
    
    y = np.split(y_true,[6])
    image_true = y[1].reshape(224,224,3)
    
    # Contrast
    contrast_mean = np.mean(image_pred)
    contrast_degenerate = np.zeros(image_pred.shape)
    
    image_pred = parameters[0] * image_pred + (1-parameters[0])*contrast_degenerate
    image_pred = np.clip(image_pred,0,1)
    
    # Brightness
    brightness_degenerate = np.zeros(image_pred.shape)
    
    image_pred = parameters[1] * image_pred + (1 - parameters[1]) * brightness_degenerate
    image_pred = np.clip(image_pred,0,1)
    
    # Color Saturation
    saturation_degenerate = image_pred.mean(axis= 2)
    
    image_pred[:,:,0] = parameters[2] * image_pred[:,:,0] + (1-parameters[2]) * saturation_degenerate
    image_pred[:,:,1] = parameters[2] * image_pred[:,:,1] + (1-parameters[2]) * saturation_degenerate
    image_pred[:,:,2] = parameters[2] * image_pred[:,:,2] + (1-parameters[2]) * saturation_degenerate
    
    # White Balance
    image_pred[:,:,0] = parameters[3] * image_pred[:,:,0]
    image_pred[:,:,1] = parameters[4] * image_pred[:,:,1]
    image_pred[:,:,2] = parameters[5] * image_pred[:,:,2]
    
    image_pred = np.clip(image_pred,0,1)
    
    mse = (np.square(image_pred - image_true)).mean(axis=None)
    
    return mse
"""
    
    
    
    
    
myNN = NeuralNetwork()

        

ValueError: Shape must be rank 1 but is rank 2 for 'loss_2/concatenate_8_loss/Slice' (op: 'Slice') with input shapes: [?,?], [1], [1].

In [7]:
myNN.combined.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 4096)         134260544   input_image[0][0]                
__________________________________________________________________________________________________
input_hist (InputLayer)         (None, 1000)         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 5096)         0           model_1[1][0]                    
                                                                 input_hist[0][0]                 
__________

In [101]:
#myNN = NeuralNetwork(224,224)

In [106]:
image_weight = 224
image_height = 224

image_size = (image_weight, image_height,3)
hist_size = 1000
batch_size = 4

optimizer = Adam(0.0002, 0.2)

input_image = Input(shape = image_size, name = 'input_image')


model_build_VGG = VGG16(weights = 'imagenet')
layer_name = 'fc2'
model_build_VGG_ = Model(inputs=model_build_VGG.input, outputs=model_build_VGG.get_layer(layer_name).output)
model_build_VGG_.trainable = False

feature_NN = model_build_VGG_(input_image)



#input_hist = Input(shape = (hist_size,), name='input_hist')
#x_ = keras.layers.concatenate([feature_NN, input_hist])




#input_hist = Input(shape = hist_size, name='input_hist')
#x = keras.layers.concatenate([feature_NN, input_hist])
#parameter_NN = build_NN()(x)

#z = Input(shape = (image_size))(input_image)
#output = keras.layer.concatenate([z, parameters_NN])

#combined = Model([input_image, input_hist], output)

In [107]:
myf = Flatten()(input_image)

x_ = keras.layers.concatenate([feature_NN, input_hist])
inputs = Input(shape = (5096,))
x = Dense(250, activation = 'relu')(inputs)
x = Dense(100, activation = 'relu')(x)
parameters = Dense(6, activation = 'linear')(x)
model2 = Model(inputs=inputs, outputs=parameters)

parameter_NN = model2(x_)
#z = Input(shape = image_size)
#z = z(input_image)
output = keras.layers.concatenate([myf, parameter_NN])

combined = Model([input_image, input_hist], output)

In [66]:
mymodel = Model(inputs = [input_hist, input_image], outputs = x)

In [76]:
parameter_NN.summary()
model2.summary()

AttributeError: 'Tensor' object has no attribute 'summary'

In [165]:
y_true = mydata.input_image[0]
np.array(y_true).shape

(224, 224, 3)

In [140]:
mydata.input_image

[array([[[[ -89.939   ,  -98.779   , -104.68    ],
          [ -82.939   ,  -91.779   ,  -97.68    ],
          [ -75.939   ,  -84.779   ,  -90.68    ],
          ...,
          [ -29.939003,  -46.779   ,  -58.68    ],
          [ -28.939003,  -47.779   ,  -61.68    ],
          [ -29.939003,  -48.779   ,  -62.68    ]],
 
         [[ -89.939   ,  -98.779   , -104.68    ],
          [ -85.939   ,  -94.779   , -100.68    ],
          [ -80.939   ,  -89.779   ,  -95.68    ],
          ...,
          [ -28.939003,  -45.779   ,  -57.68    ],
          [ -30.939003,  -47.779   ,  -59.68    ],
          [ -29.939003,  -48.779   ,  -62.68    ]],
 
         [[ -78.939   ,  -87.779   ,  -93.68    ],
          [ -80.939   ,  -89.779   ,  -95.68    ],
          [ -82.939   ,  -91.779   ,  -97.68    ],
          ...,
          [ -30.939003,  -47.779   ,  -59.68    ],
          [ -32.939003,  -49.779   ,  -61.68    ],
          [ -34.939003,  -51.779   ,  -63.68    ]],
 
         ...,
 
         [[ 

In [80]:
input_image = Input(shape = image_size, name = 'input_image')

In [92]:
myf

AttributeError: 'Tensor' object has no attribute 'layer'

In [100]:
from keras.utils import plot_model
plot_model(combined)

In [9]:
pwd

u'/home/alumno04/dev_samir/image_processing_project'

In [166]:
y_pred = K.variable(np.arange (27))
#a = np.split(y_pred,[6])
#b = a[0]
#c = a[1]
x = K.slice(y_pred, [1],[6])
y = K.slice(y_pred, [6],[-1])
z = K.reshape(y_pred,[3,3,3])

In [167]:
w = K.slice(z, [2,0,0],[1,-1,-1])
v = K.mean(w)

In [169]:
K.eval(w)

array([[[18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.]]], dtype=float32)

In [ ]:
def _loss_f(y_true, y_pred):
    x = K.slice(y_pred, [6],[-1])
        
    parameters = K.slice(y_pred,[0],[-1])
    
    image_pred = K.reshape(x,[224,224,3])
    
    image_true = K.slice(y_true,[6],[-1])
    
    # Contrast
    contrast_mean = K.mean(image_pred)
    contrast_degenerate = K.zeros(224,224,3) + contrast_mean
    
    image_pred = parameters[0] * image_pred + (1-parameters[0])*contrast_degenerate
    image_pred = K.clip(image_pred,0,1)

array([], shape=(0, 6), dtype=float32)

In [171]:
y_pred = K.variable(np.arange(150534))
y_true = K.variable(np.arange(150534))

In [270]:
def _loss_f(y_true, y_pred):
    
        
    parameter_1 = K.slice(y_pred,[0],[1])
    parameter_2 = K.slice(y_pred,[1],[1])
    parameter_3 = K.slice(y_pred,[2],[1])
    parameter_4 = K.slice(y_pred,[3],[1])
    parameter_5 = K.slice(y_pred,[4],[1])
    parameter_6 = K.slice(y_pred,[5],[1])
    
    x = K.slice(y_pred, [6],[-1])
    
    image_pred = K.reshape(x,[224,224,3])
    
    x = K.slice(y_true,[6],[-1])
    
    image_true = K.reshape(x,[224,224,3])
    
    # Contrast
    contrast_mean = K.mean(image_pred)
    contrast_degenerate = K.zeros([224,224,3]) + contrast_mean
    
    image_pred = parameter_1 * image_pred + (1-parameter_1)*contrast_degenerate
    image_pred = K.clip(image_pred,0,1)
    
     # Brightness
    #brightness_degenerate = np.zeros(image_pred.shape)
    
    image_pred = parameter_2 * image_pred # + (1 - parameters[1]) * brightness_degenerate
    image_pred = K.clip(image_pred,0,1)
    
    # Color Saturation
    
    saturation_degenerate = K.mean(image_pred,2)
    
    red_layer = K.slice(image_pred,[0,0,0],[-1,-1,1])
    blue_layer = K.slice(image_pred,[0,0,1],[-1,-1,1])
    green_layer = K.slice(image_pred,[0,0,2],[-1,-1,1])
    
    red_layer_ = (parameter_3 * red_layer) + K.expand_dims( (1- parameter_3) * saturation_degenerate, axis= 2)
    blue_layer_ = (parameter_3 * blue_layer) +  K.expand_dims( (1- parameter_3) * saturation_degenerate, axis= 2)
    green_layer_ = (parameter_3 * green_layer) +  K.expand_dims( (1- parameter_3) * saturation_degenerate, axis= 2)
    
    image_pred = K.concatenate ([red_layer_, blue_layer_, green_layer_], axis =2)
    
    image_pred = K.clip(image_pred,0,1)
    
    # White Balance
    red_layer = K.slice(image_pred,[0,0,0],[-1,-1,1])
    blue_layer = K.slice(image_pred,[0,0,1],[-1,-1,1])
    green_layer = K.slice(image_pred,[0,0,2],[-1,-1,1]) 
    
    red_layer_ = (parameter_4 * red_layer) 
    blue_layer_ = (parameter_5 * blue_layer)
    green_layer_ = (parameter_6 * green_layer)
    
    image_pred = K.concatenate ([red_layer_, blue_layer_, green_layer_], axis =2)
    
    image_pred = K.clip(image_pred,0,1)
    
    mse = K.mean(K.square(image_pred - image_true))
    
    return  mse 
    

In [271]:
K.eval(_loss_f(y_true,y_pred))

7553571000.0